In [1]:
import pandas as pd
import numpy as np

In [42]:
page1 = pd.read_html("https://www.mlb.com/stats/regular-season")
url = "https://www.mlb.com/stats/regular-season?page=2"
mlb = pd.DataFrame()
while len(pd.read_html(url)[0])>0:
    mlb = pd.concat([mlb, pd.read_html(url)[0]])
    num = int(url[-1])
    num +=1
    url = url.replace(url[-1],str(num))
mlb = pd.concat([page1[0], mlb])
mlb["Year"] = 2020
mlb

,PLAYER,TEAM,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,AVG,OBP,SLG,caret-upcaret-downOPS,Year
0,1JuanJ SotoSotoLF1‌‌‌,WSH,47,154,39,54,14,0,13,37,41,28,6,2,0.351,0.490,0.695,1.185,2020
1,2FreddieF FreemanFreeman1B2‌‌‌,ATL,60,214,51,73,23,1,13,53,45,37,2,0,0.341,0.462,0.640,1.102,2020
2,3MarcellM OzunaOzunaDH3‌‌‌,ATL,60,228,38,77,14,0,18,56,38,60,0,0,0.338,0.431,0.636,1.067,2020
3,4DJD LeMahieuLeMahieu2B4‌‌‌,NYY,50,195,41,71,10,2,10,27,18,21,3,0,0.364,0.421,0.590,1.011,2020
4,5JoseJ RamírezRamirez3B5‌‌‌,CLE,58,219,45,64,16,1,17,46,31,43,10,3,0.292,0.386,0.607,0.993,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,138EduardoE EscobarEscobar3B138‌‌‌,ARI,54,203,22,43,7,3,4,20,15,41,1,0,0.212,0.270,0.335,0.605,2020
13,139JavierJ BáezBaezSS139‌‌‌,CHC,59,222,27,45,9,1,8,24,7,75,3,0,0.203,0.238,0.360,0.599,2020
14,140EvanE WhiteWhite1B140‌‌‌,SEA,54,182,19,32,7,0,8,26,18,84,1,2,0.176,0.252,0.346,0.599,2020
15,141JonathanJ VillarVillar2B141‌‌‌,TOR,52,185,13,43,5,0,2,15,19,54,16,5,0.232,0.301,0.292,0.593,2020


In [44]:
"https://www.mlb.com/stats/2019/regular-season"

years = np.arange(1990,2020)

for year in years:
    page1 = pd.read_html("https://www.mlb.com/stats/"+str(year)+"/regular-season")
    page1 = page1[0]
    page1["Year"] = year
    url = "https://www.mlb.com/stats/"+str(year)+"?page=2"
    test = pd.DataFrame()
    while len(pd.read_html(url)[0])>0:
        test = pd.concat([test, pd.read_html(url)[0]])
        num = int(url[-1])
        num +=1
        url = url.replace(url[-1],str(num))
    test["Year"] = year
    test = pd.concat([page1,test])
    mlb = pd.concat([mlb,test])
    
    
#https://www.mlb.com/stats/2019?page=2

In [53]:
mlb

,PLAYER,TEAM,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,AVG,OBP,SLG,caret-upcaret-downOPS,Year
0,1JuanJ SotoSotoLF1‌‌‌,WSH,47,154,39,54,14,0,13,37,41,28,6,2,0.351,0.490,0.695,1.185,2020
1,2FreddieF FreemanFreeman1B2‌‌‌,ATL,60,214,51,73,23,1,13,53,45,37,2,0,0.341,0.462,0.640,1.102,2020
2,3MarcellM OzunaOzunaDH3‌‌‌,ATL,60,228,38,77,14,0,18,56,38,60,0,0,0.338,0.431,0.636,1.067,2020
3,4DJD LeMahieuLeMahieu2B4‌‌‌,NYY,50,195,41,71,10,2,10,27,18,21,3,0,0.364,0.421,0.590,1.011,2020
4,5JoseJ RamírezRamirez3B5‌‌‌,CLE,58,219,45,64,16,1,17,46,31,43,10,3,0.292,0.386,0.607,0.993,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,113MikeM MoustakasMoustakas2B113‌‌‌,CIN,2,8,0,0,0,0,0,0,1,2,0,0,0.000,0.111,0.000,0.111,2019
13,113CarlosC SantanaSantana1B113‌‌‌,CLE,2,8,1,0,0,0,0,0,1,3,0,0,0.000,0.111,0.000,0.111,2019
14,115KrisK BryantBryant3B115‌‌‌,CHC,2,8,0,0,0,0,0,0,0,2,0,0,0.000,0.000,0.000,0.000,2019
15,115AnthonyA RizzoRizzo1B115‌‌‌,CHC,2,8,0,0,0,0,0,0,0,3,0,0,0.000,0.000,0.000,0.000,2019
